In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import random
# from selenium import webdriver

headers = { 
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36', 
'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 
'Accept-Language' : 'en-US,en;q=0.5',
'Accept-Encoding' : 'gzip', 
'DNT' : '1', # Do Not Track Request Header 
'Connection' : 'close'
}
# headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
# headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 5.1.1; SM-G928X Build/LMY47X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36'} 

### Download DataSET  of URL's from [here](https://github.com/taruntiwarihp/dataSets/blob/master/Amazon%20Monthly%20URLs.xlsx)

In [3]:
def setup(i):
    r = requests.get(i, headers=headers)#, proxies=proxies)
    content = r.content
    soupt = BeautifulSoup(content)
    return soupt

In [4]:
def prod_details(soupt):
    ### Product Name ###
    prod_name = soupt.find("span",attrs={"class":"a-size-large product-title-word-break"})
    p_name = prod_name.text
    prod_name = p_name.strip()

    ### Rating Star ###
    rsoup = soupt.find("span",attrs={"class":"a-icon-alt"})
    rating_text = rsoup.text   

    ### Total Rating ###
    btsoup = soupt.find("div",attrs={"class":"centerColAlign centerColAlign-bbcxoverride"})
    rtsoup = btsoup.find("span",attrs={"class":"a-size-base"})
    total_rating = rtsoup.text
    
    prod_d = [prod_name, rating_text, total_rating]
    return prod_d

In [5]:
def reviews_l(soup):
    link = soup.find("a",attrs={"class":"a-link-emphasis a-text-bold"})
#     link = soup.find("div",attrs={"class":"a-section"})
    main = 'https://www.amazon.com'
#     print(link.find('a').get("href"))
    review_link = main+link.get("href") #find('a').get("href")
    r = requests.get(review_link, headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content,"lxml")
    deckk = soup.find("div",attrs={"class","a-section a-spacing-none review-views celwidget"})
    return deckk

In [6]:
def details_s(deck):
    ### Names ###
    name = deck.find_all("span",attrs={"class","a-profile-name"})
    names = []
    for n in name:
        names.append(n.text)
    
    ### Ratings ###
    rr, rating =[],[]
    r = deck.find_all("a",attrs={"class","a-link-normal"})

    for rk in r:
        rr.append(rk.text)

    c,x,y = 0,0,1

    for _ in range(len(rr)):
        grr = rr[x:y]
        rating.append(', '.join(grr))
        x+=4
        y+=4
        c+=1
        if c==10:
            break

    ### Titles ###
    titles = []
    t = deck.find_all("a",attrs={"class","a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"})
    for i in t:
        titles.append(i.find("span").text)

    ### Dates ###
    dates=[]
    date = deck.find_all("span",attrs={"class":"a-size-base a-color-secondary review-date"})
    for i in range(len(date)):
        gdate = date[i].text
        dates.append(gdate[33:])

    ### Verifies ###    
    verifies = []
    verifie = deck.find_all("span",attrs={"class":"a-size-mini a-color-state a-text-bold"})
    for i in range(len(verifie)):
        verifies.append(verifie[i].text)

    ### Comment ###
    story,comment=[],[]
    s = deck.find_all("span",attrs={"class","a-size-base review-text review-text-content"})
    for ss in s:
        story.append(ss.find("span").text.strip())

    x, y = 0, 1
    for _ in range(len(story)):
        cs = story[x:y]
        comment.append(', '.join(cs))
        x += 1
        y += 1
    
    revieww =[titles,rating,names,dates,comment,verifies,]
    return revieww

In [7]:
def next_r_link(decks):
    flag=1
    deck = decks
    lk = deck.find("li",attrs={"class","a-last"})
#     print(lk)
    if lk == None:
        return flag
    mk = lk.attrs
    tt = list(mk.values())
    if tt[0][0] != 'a-last': #"a-disabled a-last":
        return flag
    main = 'https://www.amazon.com'
    lk = lk.find("a")
    nlk = main+str(lk.get('href'))
    r = requests.get(nlk, headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    deckk = soup.find("div",attrs={"class","a-section a-spacing-none review-views celwidget"})
    return deckk

In [8]:
def start(soupt):
    frames=[]
    soup=soupt
    col = ['review_title','StarRating','reviewer_name','review_date','review_description','Other_Tags' ]
    decks = reviews_l(soupt)
    while(True):
        ### first frame ###
        review = details_s(decks)
        review[1][0:1]
        reviews = pd.DataFrame(review)
        reviews = reviews.T
        reviews.columns = col
        frames.append(reviews)
        decks = next_r_link(decks)
        if decks == 1:
            df = pd.concat(frames)
            return df
        else:
            continue

In [16]:
dfx = pd.read_excel("Amazon Monthly URLs.xlsx")
list_urls = dfx['URL']

In [17]:
list_urls = list_urls[1:2]

In [18]:
print(list_urls)

1    http://www.amazon.com/dp/B00002N6WX
Name: URL, dtype: object


In [19]:
# dfx = pd.read_excel("Amazon Monthly URLs.xlsx")
# dfx.drop([25,26,27,28,29,30,31,32,33,34,35],inplace=True) # 25,29
# dfx.shape
final = []
mm = []
for idx, i in enumerate(list_urls): 
    print(idx,i)
    sp = setup(i)
#     print(sp)
    df = start(sp)
    l = prod_details(sp)
    df.reset_index(drop=True, inplace=True)
    reviews = df
    print(idx,i[25:])
    reviews['input_url'] = str(i)
    reviews['asin'] = i[25:]
    reviews['product_name'] = l[0]
    reviews['overall_stars_rating'] = l[1]
#     reviews['total_reviews'] = int(float(mm))
    reviews['monthly_reviews_date'] = '1/11/2020'
    reviews['monthly_reviews_count'] = 0
    reviews.dropna(inplace=True)
    one = reviews[reviews['review_date'].str.contains('2020')]
    one = one[one['review_date'].str.contains('Novemver ')]
    
#     if len(one) != 0:
#         mm.append(l[2][:-8])
#         one['monthly_reviews_count'] = total_reviews[0]-len(one)
#     final.append(one)
        

0 http://www.amazon.com/dp/B00002N6WX
0 B00002N6WX


In [20]:
reviews.head()

,review_title,StarRating,reviewer_name,review_date,review_description,Other_Tags,input_url,asin,product_name,overall_stars_rating,monthly_reviews_date,monthly_reviews_count
0,"High quality appearance and function, horizont...",5.0 out of 5 stars,Bob in VA,"January 20, 2014","As usual with Baldwin products, this is a high...",Verified Purchase,http://www.amazon.com/dp/B00002N6WX,B00002N6WX,Baldwin Estate 8231.003 Low Profile Traditiona...,4.3 out of 5 stars,1/11/2020,0
1,Beautiful lock!,5.0 out of 5 stars,WoodButcher,"January 31, 2019",I replaced a 20+ year-old Baldwin lock. Compl...,Verified Purchase,http://www.amazon.com/dp/B00002N6WX,B00002N6WX,Baldwin Estate 8231.003 Low Profile Traditiona...,4.3 out of 5 stars,1/11/2020,0
2,Five Stars,5.0 out of 5 stars,John Doe,"March 19, 2017",very nice,Verified Purchase,http://www.amazon.com/dp/B00002N6WX,B00002N6WX,Baldwin Estate 8231.003 Low Profile Traditiona...,4.3 out of 5 stars,1/11/2020,0
3,Value,5.0 out of 5 stars,Frenchie,"November 24, 2012",You cannot beat solid brass for external doors...,Verified Purchase,http://www.amazon.com/dp/B00002N6WX,B00002N6WX,Baldwin Estate 8231.003 Low Profile Traditiona...,4.3 out of 5 stars,1/11/2020,0
4,Very nice,5.0 out of 5 stars,Sheila Jones,"December 8, 2012",I purchased these deadbolt locks to go with th...,Verified Purchase,http://www.amazon.com/dp/B00002N6WX,B00002N6WX,Baldwin Estate 8231.003 Low Profile Traditiona...,4.3 out of 5 stars,1/11/2020,0


In [21]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   review_title           14 non-null     object
 1   StarRating             14 non-null     object
 2   reviewer_name          14 non-null     object
 3   review_date            14 non-null     object
 4   review_description     14 non-null     object
 5   Other_Tags             14 non-null     object
 6   input_url              14 non-null     object
 7   asin                   14 non-null     object
 8   product_name           14 non-null     object
 9   overall_stars_rating   14 non-null     object
 10  monthly_reviews_date   14 non-null     object
 11  monthly_reviews_count  14 non-null     int64 
dtypes: int64(1), object(11)
memory usage: 1.4+ KB


### Follow me on Github for more Updates